# 先Filter 在2015/12 ~ 2018/12 存續的ETF

## 先開啟檔案位置，並設定初始時間作為篩選門檻

In [2]:
import csv
from datetime import datetime

start_date = datetime.strptime( "2016/01/01" , "%Y/%m/%d")

out_file =  open( 'ETF List Filtered.csv', 'w',newline='')
in_file1 = open( '..\data\Currency ETF List (36).csv', 'r') 
in_file2 = open( '..\data\Preferred Stock ETF List (12).csv', 'r')

## 使用CSV庫，用DictReader來讀取特定的行數，用法類似字典，並用writer來寫出檔案

In [4]:
csv_reader = csv.DictReader(in_file1)
csv_writer = csv.writer(out_file)
csv_writer.writerow(['Symbol', 'ETF Name', 'Inception','Type']) #先寫出行名稱

for a_row in csv_reader: #對每一列，進行Dictionary-like的讀取
    symbol = a_row['Symbol']
    etf_name = a_row['ETF Name']
    inception = datetime.strptime( a_row['Inception'], "%Y/%m/%d") 
#寫入
    if inception  <= start_date:
        csv_writer.writerow([symbol, etf_name, inception.date(), 'Currency']) #寫出四行，分別是代號、名稱、創立時間、跟基金類型

In [5]:
csv_reader2 = csv.DictReader(in_file2)
for a_row in csv_reader2:
    symbol = a_row['Symbol']
    etf_name = a_row['ETF Name']
    inception = datetime.strptime( a_row['Inception'], "%Y/%m/%d")
#寫入
    if inception  <= start_date:
        csv_writer.writerow([symbol, etf_name, inception.date(), 'Prefered Stock'])

## 記得關閉檔案

In [6]:
out_file.close()
in_file1.close()
in_file2.close()

# selenium

## 導入selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC    

## 設定driver

In [2]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)


## 定義抓取首頁的函數

In [3]:
def get_home_page(symbol):
    url = "https://www.etf.com/" + symbol
    r = requests.get(url)
    r.encoding = 'utf-8'
    soup = bs(r.text, 'lxml')	#'lxml'解析器	優點速度快
    pattern = soup.find('div',attrs={"class": "field-content helplink"}) #首頁鏈結放在該標籤之下
    url2 = pattern.a['href']
    return url2

In [4]:
get_home_page('FXY')

'https://www.invesco.com/portal/site/us/financial-professional/etfs/product-detail?productId=FXY&ticker=FXY&title=powershares-currencyshares-japanese-yen-trust'

## 接著需要知道每個基金公司把NAV淨值放在何處，以便我們用Selenium來點取下載鏈接
## 這部份靠人工去尋找，接下來只需要利用Katalon Recoder 來生成python 指令即可
## 先產生csv file reader

In [5]:
import csv
path = 'ETF List Filtered.csv'
in_file =  open( path , 'r')
csv_reader = csv.DictReader(in_file)

## 如果有些下載鍵結可以用pandas直接打開，或者用requests下載，那就省去一個步驟

In [6]:
import pandas as pd
import time
for a_row in csv_reader:
    symbol = a_row['Symbol']
    etf_name = a_row['ETF Name']
    company_name = etf_name.split()[0]
    if company_name == 'Invesco':  #如果基金公司是Invesco，那下載鏈結為id = downloadNavHistory
        url = get_home_page(symbol)
        driver.get(url)
        driver.find_element_by_id("downloadNavHistory").click()
    elif company_name == 'Market': ##如果基金公司是Market Vectors，可直接用網址加代號取得檔案
        driver.get('https://www.marketvectorsetns.com/HistoPriceExport.aspx?ticker=' + symbol)
        
    elif company_name == 'ProShares': ##如果基金公司是ProShares，可直接用網址加代號取得CSV檔案  
        url = 'https://accounts.profunds.com/etfdata/ByFund/' + symbol +'-historical_nav.csv'
        df = pd.read_csv(url)
        df.to_csv( 'C:\\Users\\User\\Downloads\\%s.csv' %symbol)
        
    elif company_name == 'WisdomTree': ##如果基金公司是WisdomTree，得先進去一個網址後用beautifulsoup以及pandas來處理網站表格
        homepage = 'https://www.wisdomtree.com/etfs/currency/'+ symbol.lower()
        r = requests.get(homepage)
        r.encoding = 'utf-8'
        soup = bs(r.text, 'lxml')
        pattern = soup.find('ul',attrs={"class": "footer-links"}) 
        table_html = pattern.a['data-href']

        r = requests.get(table_html)
        html_df = pd.read_html(r.text)
        html_df[0].to_csv( 'C:\\Users\\User\\Downloads\\%s.csv' %symbol)
    elif company_name == 'SPDR': ##如果基金公司是SPDR，得先進去一個網址後用beautifulsoup以及pandas來處理網站表格
        driver.get('https://us.spdrs.com/site-content/xls/PSK_HistoricalNav.xls?fund=' + symbol)
        
    else:#其他基金公司 必須從yahoo來爬取
        url = 'https://finance.yahoo.com/quote/' + symbol + "/history?period1=1448899200&period2=1546272000&interval=1d&filter=history&frequency=1d"
        driver.get(url)
        time.sleep(2)
        wait.until(EC.presence_of_element_located((By.XPATH, "(.//*[normalize-space(text()) and normalize-space(.)='Currency in USD'])[1]/following::span[2]")))
        driver.find_element_by_xpath("(.//*[normalize-space(text()) and normalize-space(.)='Currency in USD'])[1]/following::span[2]").click()
    print('Symbol %s is done.' %symbol)

Symbol FXA is done.
Symbol FXB is done.
Symbol FXC is done.
Symbol FXCH is done.
Symbol FXE is done.
Symbol FXY is done.
Symbol FXSG is done.
Symbol FXS is done.
Symbol FXF is done.
Symbol DBV is done.
Symbol UDN is done.
Symbol UUP is done.
Symbol URR is done.
Symbol DRR is done.
Symbol CNY is done.
Symbol INR is done.
Symbol EUFX is done.
Symbol ULE is done.
Symbol YCL is done.
Symbol CROC is done.
Symbol EUO is done.
Symbol YCS is done.
Symbol USDU is done.
Symbol BZF is done.
Symbol CYB is done.
Symbol CEW is done.
Symbol FPE is done.
Symbol SPFF is done.
Symbol PGF is done.
Symbol PGX is done.
Symbol IPFF is done.
Symbol PFF is done.
Symbol PSK is done.
Symbol PFXF is done.


## 接著我們要合併這些檔案，在此之前，一個簡單的例子來說明如何合併dataframe

In [7]:
dict1 ={'12/1' : 23, '12/2': 24, '12/3': 22} 
dict2 ={'12/1' : 233, '12/2': 234, '12/3': 322} 



import pandas as pd
df1 = pd.DataFrame.from_dict(dict1, orient='index',  columns=['FXY'])
df2 = pd.DataFrame.from_dict(dict2, orient='index',  columns=['FXA'])

res = pd.concat([df1, df2], axis=1)
print( res)

      FXY  FXA
12/1   23  233
12/2   24  234
12/3   22  322


## 先找到檔案資料夾，並找出所有剛剛下載的檔案的檔名

In [8]:
import os
import glob

path = r'C:\Users\User\data'	#the path where you put your downloaded files
extension1 = 'csv'
extension2 = 'xls'
os.chdir(path)
csv_list = [i for i in glob.glob('*.{}'.format(extension1))]
excel_list = [i for i in glob.glob('*.{}'.format(extension2))]
print(csv_list)
print(excel_list)

['BZF.csv', 'CEW.csv', 'CROC.csv', 'CYB.csv', 'EUFX.csv', 'EUO.csv', 'FPE.csv', 'historical_navs_DBV_1552797335902.csv', 'historical_navs_FXA_1552797302307.csv', 'historical_navs_FXB_1552797305703.csv', 'historical_navs_FXCH_1552797313630.csv', 'historical_navs_FXC_1552797310008.csv', 'historical_navs_FXE_1552797317332.csv', 'historical_navs_FXF_1552797332125.csv', 'historical_navs_FXSG_1552797324760.csv', 'historical_navs_FXS_1552797328470.csv', 'historical_navs_FXY_1552797320997.csv', 'historical_navs_pgf_1552797398016.csv', 'historical_navs_pgx_1552797402471.csv', 'historical_navs_UDN_1552797339536.csv', 'historical_navs_UUP_1552797343168.csv', 'IPFF.csv', 'PFF.csv', 'PFXF.csv', 'SPFF.csv', 'ULE.csv', 'USDU.csv', 'YCL.csv', 'YCS.csv']
['CNY_asof_20190317.xls', 'DRR_asof_20190317.xls', 'INR_asof_20190317.xls', 'PSK_HistoricalNav.xls', 'URR_asof_20190317.xls']


## 先看一下excel檔案 如何處理
## 注意!! 有些檔案無法用read_excel讀取，必須要read_html來讀取，否則會出錯

In [20]:

for file in excel_list:
	print(file)
	try:
		df = pd.read_html(path + '\\' +file, skiprows=2)[0].iloc[:, 0:2]
		print(df.head())
	except:
		df = pd.read_excel(path + '\\' +file, skiprows=3).iloc[:-12, 0:2]	#該檔案最後幾行是文字，而非我們需要的
		print(df.head())
		

CNY_asof_20190317.xls
           0        1
0  3/15/2019  44.8950
1  3/14/2019  44.6000
2  3/13/2019  44.2893
3  3/12/2019  44.4300
4  3/11/2019  44.4200
DRR_asof_20190317.xls
           0       1
0  3/15/2019  59.775
1  3/14/2019  61.010
2  3/13/2019  60.425
3  3/12/2019  59.845
4  3/11/2019  60.070
INR_asof_20190317.xls
           0        1
0  3/15/2019  42.9999
1  3/14/2019  42.3950
2  3/13/2019  41.5100
3  3/12/2019  41.0500
4  3/11/2019  41.0000
PSK_HistoricalNav.xls
          Date        Nav
0  14-Mar-2019  42.534408
1  13-Mar-2019  42.472707
2  12-Mar-2019  42.383852
3  11-Mar-2019  42.334389
4  08-Mar-2019  42.357835
URR_asof_20190317.xls
           0      1
0  3/15/2019  16.55
1  3/14/2019  16.55
2  3/13/2019  16.55
3  3/12/2019  16.55
4  3/11/2019  16.55


## 看一下csv如何處理

In [14]:
for file in csv_list:
	df = pd.read_csv(path + '\\' +file)
	col_name = list(df.columns.values)
	print(file, col_name)

BZF.csv ['Unnamed: 0', 'Date', 'Nav', 'Mid Bid/Ask', 'P/D to NAV', 'P/D Indicator']
CEW.csv ['Unnamed: 0', 'Date', 'Nav', 'Mid Bid/Ask', 'P/D to NAV', 'P/D Indicator']
CROC.csv ['Unnamed: 0', 'Date', 'ProShares Name', 'Ticker', 'NAV', 'Prior NAV', 'NAV Change (%)', 'NAV Change ($)', 'Shares Outstanding (000)', 'Assets Under Management']
CYB.csv ['Unnamed: 0', 'Date', 'Nav', 'Mid Bid/Ask', 'P/D to NAV', 'P/D Indicator']
EUFX.csv ['Unnamed: 0', 'Date', 'ProShares Name', 'Ticker', 'NAV', 'Prior NAV', 'NAV Change (%)', 'NAV Change ($)', 'Shares Outstanding (000)', 'Assets Under Management']
EUO.csv ['Unnamed: 0', 'Date', 'ProShares Name', 'Ticker', 'NAV', 'Prior NAV', 'NAV Change (%)', 'NAV Change ($)', 'Shares Outstanding (000)', 'Assets Under Management']
FPE.csv ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
historical_navs_DBV_1552797335902.csv ['Ticker', 'NAV', 'Date']
historical_navs_FXA_1552797302307.csv ['Ticker', 'NAV', 'Date']
historical_navs_FXB_1552797305703.cs

## 有些檔案的NAV 是寫作 'Nav' 或者 'Adj Close'，需要注意

In [17]:
for file in csv_list:
	df = pd.read_csv(path + '\\' +file)
	col_name = list(df.columns.values)
	if 'NAV' not in col_name:
		if 'Nav' in col_name:
			df = df.loc[:, ['Date', 'Nav']]
		else:
			df = df.loc[:, ['Date', 'Adj Close']]
	
			
	else:
		df = df.loc[:, ['Date', 'NAV']]
	print(file)
	print(df.head())

BZF.csv
         Date      Nav
0  03/14/2019  16.5367
1  03/13/2019  16.5789
2  03/12/2019  16.6396
3  03/11/2019  16.4877
4  03/08/2019  16.4171
CEW.csv
         Date      Nav
0  03/15/2019  18.4994
1  03/14/2019  18.4505
2  03/13/2019  18.4614
3  03/12/2019  18.4708
4  03/11/2019  18.4207
CROC.csv
         Date        NAV
0  03/15/2019  54.627192
1  03/14/2019  54.936440
2  03/13/2019  54.472166
3  03/12/2019  54.639856
4  03/11/2019  54.862680
CYB.csv
         Date      Nav
0  03/15/2019  26.0629
1  03/14/2019  26.0879
2  03/13/2019  26.1014
3  03/12/2019  26.0751
4  03/11/2019  26.0375
EUFX.csv
         Date        NAV
0  03/15/2019  43.940353
1  03/14/2019  44.007569
2  03/13/2019  43.876602
3  03/12/2019  44.017162
4  03/11/2019  44.205642
EUO.csv
         Date        NAV
0  03/15/2019  25.135626
1  03/14/2019  25.215577
2  03/13/2019  25.070370
3  03/12/2019  25.228045
4  03/11/2019  25.448762
FPE.csv
         Date  Adj Close
0  2015-11-30  15.811456
1  2015-12-01  15.811456
2  

## 開始處理資料
## 要注意，由於Date 除了'03/15/2019'("%m/%d/%Y")方式呈現外，還有'14-Mar-2019'("%d-%b-%Y")及2015-11-30("%Y-%m-%d")呈現，需要處理他們

In [23]:
from datetime import datetime
df_list = []
start_of_2016 =  datetime.strptime( '1/1/2016', "%m/%d/%Y")	
		
for file in excel_list:
	try:
		df = pd.read_html(path + '\\' +file, skiprows=2)[0].iloc[:, 0:2]
		
	except:
		df = pd.read_excel(path + '\\' +file, skiprows=3).iloc[:-12, 0:2]	#該檔案最後幾行是文字，而非我們需要的
			
	#使用dictionary來處理資料
	tem_dict = {}
	dict = df.to_dict('split')
	list_of_date_and_nav = dict['data']		

	#開始轉成dictionary，需要調整時間格式!!
	#還需要確保此時間不會早於2015年底的最後一個交易日
	max_date_prior_to_2016 = datetime.strptime( '12/1/2015', "%m/%d/%Y")	#時間的default value，為了找出2015年的最後一個交易日為何時。
	for date, nav in list_of_date_and_nav:
		try:
			datetime = datetime.strptime( date, "%d-%b-%Y")
		except:
			datetime = datetime.strptime( date, "%m/%d/%Y")	
	
		if datetime >= start_of_2016:
			tem_dict[datetime] = nav
		elif start_of_2016 > datetime > max_date_prior_to_2016:	#更新2015年的最後一個交易日
			if max_date_prior_to_2016 in tem_dict:
				tem_dict.pop(max_date_prior_to_2016)
			tem_dict[datetime] = nav
			max_date_prior_to_2016 = datetime
		
	#找出symbol 來當作 column name，並轉成dataframe
	symbol =  file.split('_')[0]
	df = pd.DataFrame.from_dict(tem_dict, orient='index',  columns=[symbol])
	
	print(df.head(1))
	print(df.tail(1))
	#append 到list 以便之後合併
	df_list.append(df)



for file in csv_list:
	#找出datafram，並留下Date及NAV
	df = pd.read_csv(path + '\\' +file)
	col_name = list(df.columns.values)
	if 'NAV' not in col_name:
		if 'Nav' in col_name:
			df = df.loc[:, ['Date', 'Nav']]
		else:
			df = df.loc[:, ['Date', 'Adj Close']]	
	else:
		df = df.loc[:, ['Date', 'NAV']]
	
	#使用dictionary來處理資料
	tem_dict = {}
	dict = df.to_dict('split')
	list_of_date_and_nav = dict['data']
	
	#開始轉成dictionary，需要調整時間格式!!
	#還需要確保此時間不會早於2015年底的最後一個交易日
	max_date_prior_to_2016 = datetime.strptime( '12/1/2015', "%m/%d/%Y")	#時間的default value，為了找出2015年的最後一個交易日為何時。
	for date, nav in list_of_date_and_nav:
		try:
			datetime = datetime.strptime( date, "%m/%d/%Y")
		except:
			datetime = datetime.strptime( date, "%Y-%m-%d")
			
		if datetime >= start_of_2016:
			tem_dict[datetime] = nav
		elif datetime > max_date_prior_to_2016:	#更新2015年的最後一個交易日
			if max_date_prior_to_2016 in tem_dict:
				tem_dict.pop(max_date_prior_to_2016)
			tem_dict[datetime] = nav
			max_date_prior_to_2016 = datetime
			
	#找出symbol 來當作 column name，並轉成dataframe
	if file.split('_')[0] == 'historical':
		symbol =  file.split('_')[2]
	else:
		symbol =  file.split('.')[0]
	df = pd.DataFrame.from_dict(tem_dict, orient='index',  columns=[symbol])
	
	print(df.head(1))
	print(df.tail(1))
	#append 到list 以便之後合併
	df_list.append(df)

               CNY
2019-03-15  44.895
              CNY
2015-12-31  41.22
               DRR
2019-03-15  59.775
             DRR
2015-12-31  59.7
                INR
2019-03-15  42.9999
              INR
2015-12-31  36.87
                  PSK
2019-03-14  42.534408
                  PSK
2015-12-31  44.612338
              URR
2019-03-15  16.55
             URR
2015-12-31  17.0
                BZF
2019-03-14  16.5367
                BZF
2015-12-31  12.7654
                CEW
2019-03-15  18.4994
                CEW
2015-12-31  16.7606
                 CROC
2019-03-15  54.627192
               CROC
2015-12-31  58.5401
                CYB
2019-03-15  26.0629
                CYB
2015-12-31  24.3472
                 EUFX
2019-03-15  43.940353
               EUFX
2015-12-31  43.7807
                  EUO
2019-03-15  25.135626
                EUO
2015-12-31  25.5412
                  FPE
2015-12-31  15.822186
                  FPE
2018-12-31  17.790955
              DBV
2019-03-17  24.53
    

In [24]:
res = pd.concat( df_list , axis=1)
print( res )

                CNY      DRR      INR        PSK      URR      BZF      CEW  \
2015-12-31  41.2200  59.7000  36.8700  44.612338  17.0000  12.7654  16.7606   
2016-01-01      NaN      NaN      NaN        NaN      NaN      NaN      NaN   
2016-01-02      NaN      NaN      NaN        NaN      NaN      NaN      NaN   
2016-01-03      NaN      NaN      NaN        NaN      NaN      NaN      NaN   
2016-01-04  40.2600  60.4304  36.7000  44.779912  17.0000  12.5150  16.6179   
2016-01-05  41.1600  61.2130  36.7000  44.911751  17.0000  12.5646  16.6191   
2016-01-06  40.2320  61.0200  36.7000  44.789160  17.0000  12.5598  16.5037   
2016-01-07  40.3348  59.0105  36.8800  44.619737  17.0180  12.5528  16.4855   
2016-01-08  39.9207  59.5000  36.6700  44.753689  17.1169  12.5249  16.4484   
2016-01-09      NaN      NaN      NaN        NaN      NaN      NaN      NaN   
2016-01-10      NaN      NaN      NaN        NaN      NaN      NaN      NaN   
2016-01-11  40.7500  59.6596  36.6700  44.625197  16

In [26]:
res.to_csv('output2.csv')